In [1]:
!pip install pytorch_lightning==0.7.6
!pip install comet-ml
!pip install torchsummary

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jul  8 01:45:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
checkpoints/        pytorch-xla-env-setup.py
data/               src/
lightning_logs/     torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl
pretrained_models/  torchvision-nightly+20200325-cp36-cp36m-linux_x86_64.whl
provided_code/      torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl


In [5]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.logging import CometLogger
from src.models.pix2pix_model import Pix2PixModel
from pytorch_lightning.callbacks import ModelCheckpoint
from src.options.train_options import TrainOptions
from torchsummary import summary

In [6]:
args = ['--batchSize', '1',
        '--primary_directory', primary_directory,
        '--which_model_netG', 'resnet_unet',
        '--resnet_depth', '34',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100',
        '--lr_policy', 'cyclic',
        '--epoch_count', '200',
        '--load_epoch', '-1',
        '--lr', '0.01',
        '--lr_max', '0.005',
        '--lr_step_size', '50',
        '--init_type', 'xavier',
        '--no_perceptual_loss']

opt = TrainOptions().parse(args)

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 200
fineSize: 128
gpu_ids: -1
init_type: xavier
input_nc: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
lambda_perceptual: 10.0
loadSize: 128
load_epoch: -1
lr: 0.01
lr_decay_iters: 50
lr_max: 0.005
lr_policy: cyclic
lr_step_size: 50
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
no_perceptual_loss: True
norm: batch_3d
nwf: 64
output_nc: 1
phase: train
pool_size: 50
primary_directory: /content/drive/My Drive/KBP_Challenge
print_freq: 200
recon_loss: l1
resize_or_crop: resize_and_crop
resnet_depth: 34
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: Fals

In [8]:
model = Pix2PixModel(opt, stage='training')
summary(model.generator.to("cuda"), (1, 128, 128, 128))

loading pretrained model /content/drive/My Drive/KBP_Challenge/pretrained_models/resnet_34_23dataset.pth


/content/drive/My Drive/KBP_Challenge/src/models/resnet3d.py:166: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/KBP_Challenge/src/models/networks.py:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data, gain=0.02)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 64, 128, 128, 128]           1,792
       BatchNorm3d-2    [-1, 64, 128, 128, 128]             128
              ReLU-3    [-1, 64, 128, 128, 128]               0
            Conv3d-4    [-1, 64, 128, 128, 128]         110,656
       BatchNorm3d-5    [-1, 64, 128, 128, 128]             128
              ReLU-6    [-1, 64, 128, 128, 128]               0
            Conv3d-7       [-1, 64, 64, 64, 64]          21,952
       BatchNorm3d-8       [-1, 64, 64, 64, 64]             128
              ReLU-9       [-1, 64, 64, 64, 64]               0
        MaxPool3d-10       [-1, 64, 32, 32, 32]               0
           Conv3d-11       [-1, 64, 32, 32, 32]         110,592
      BatchNorm3d-12       [-1, 64, 32, 32, 32]             128
             ReLU-13       [-1, 64, 32, 32, 32]               0
           Conv3d-14       [-1, 64, 32,

In [9]:
checkpoints_dir = '{}/checkpoints'.format(primary_directory)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoints_dir,
    verbose=True,
    monitor='loss',
    mode='min'
)
lr_logger = LearningRateLogger()

comet_logger = CometLogger(
    api_key="eyAsnp1KA7fXLxFMkEWKjhygS",
    project_name="kbp-challenge",
    workspace="mahruskazi"
)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /content/drive/My Drive/KBP_Challenge/checkpoints exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
CometLogger will be initialized in online mode


In [ ]:
checkpoint = None
if opt.load_epoch != -1:
  checkpoint = '{}/epoch={}.ckpt'.format(checkpoints_dir, opt.load_epoch)

trainer = pl.Trainer(logger=comet_logger,
                     resume_from_checkpoint=checkpoint,
                     gpus=1,
                     checkpoint_callback=checkpoint_callback,
                     callbacks=[lr_logger],
                     max_epochs=opt.epoch_count,
                     check_val_every_n_epoch=1,
                     num_sanity_val_steps=10,
                     val_percent_check=1,
                     accumulate_grad_batches=8)
trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mahruskazi/kbp-challenge/d3b1aca4fd4d4258bae87cc243399063


    | Name                              | Type                | Params
----------------------------------------------------------------------
0   | generator                         | ResNetUNet          | 82 M  
1   | generator.resnet_layer0           | Sequential          | 22 K  
2   | generator.resnet_layer0.0         | Conv3d              | 21 K  
3   | generator.resnet_layer0.1         | BatchNorm3d         | 128   
4   | generator.resnet_layer0.2         | ReLU          

Number of validation patients: 50


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


100%|██████████| 10/10 [00:07<00:00,  1.28it/s]

{'dvh_score': 19.604800603412116, 'dose_score': 25.22946212628032}
Number of training patients: 150
Number of validation patients: 50



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
